In [ ]:
class Strategy:
    def __init__(self, data) -> None:
        self.data = data
        self.indicator = None

data = [0,1,2,3,4,5]
strat = Strategy(data)
strat.__dict__.items()

In [ ]:
print(len.__name__)
print(len.__class__.__name__)

In [ ]:
import datetime as dt 
import yfinance as yf
import pandas as pd

num_days = 730
start=dt.date(2021, 1, 1)
start = max(start, dt.date.today() - dt.timedelta(days=num_days-1))
end = dt.date.today() 
print(start, end)
data = yf.download("SPY", interval="1h", start=start, end=end, prepost=True)
data = data.drop(['Adj Close'], axis=1)
data.index = list(map(lambda x: dt.datetime.strptime(str(x).replace(":",""), '%Y-%m-%d %H%M%S%z').replace(tzinfo=None), data.index))
data = data.rename_axis("Datetime").reset_index()
print(data)


In [ ]:
timestr = '2022-07-28 04:00:00'
output = dt.datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S')
print(output)


In [ ]:
import bokeh.colors

# for c in dir(bokeh.colors.named):
    # print(c, c.__class__)
import bokeh.palettes
print(bokeh.palettes.Dark2_8)

# print(bokeh.colors.named.blue.__class__)
# print(all_colors)

# Top gainers aftermath

In [ ]:
from tradingview_screener import *
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (Query()
 .select('name', 'close', 'change', 'volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
 .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('change') > 5,
     Column('volume') > 1E6,
     Column('close') > 2,
    )
 .order_by('change', ascending=False)
 .get_scanner_data())

df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "change": "Change %",
    "float_shares_outstanding": "Float (M)"
    })
df = df[df.ticker.str.contains("NASDAQ:") | df.ticker.str.contains("NYSE:")]
print(len(df))
display(df)


In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

(dt.date(2024, 9, 1) - BDay(0)).date()

In [ ]:
import datetime as dt
from pandas.tseries.offsets import BDay

# all_data = {dt.date.today() : df.ticker.to_list()}
fake_date_range = [dt.date(2024, 9, i) for i in range(1, 30)]
fake_data = {fake_date_range[i//2] : [str(j) for j in range(i, i+3)] for i in range(0, len(fake_date_range)*2,2)}
print(fake_data)


observation_time = 1 # days after making top gainer list
for today_date in fake_date_range:
    stocks_to_scan = set()
    for time_delta in range(observation_time + 1):
        # lookback_date = (today_date - BDay(time_delta)).date()
        lookback_date = today_date - dt.timedelta(days=time_delta)
        print(lookback_date)
        if lookback_date in fake_data:
            stocks_to_scan.update(fake_data[lookback_date])
            print("updated:", fake_data[lookback_date])
    print(today_date, stocks_to_scan)
    print()
        

# Premarket

In [ ]:
from tradingview_screener import *
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

n_rows, df = (
    Scanner.premarket_gainers
    .select('name', 'close', 'volume', 'premarket_change', 'premarket_volume', 'relative_volume_10d_calc', 'market_cap_basic', 'float_shares_outstanding')
    .where(
     Column('market_cap_basic') >= 1_000_000,
     Column('relative_volume_10d_calc') > 1.2,
     Column('premarket_change') > 5,
     Column('premarket_volume') > 1E6,
     Column('close') > 1
    )
    .get_scanner_data())


df.volume = df.volume * 1E-6
df.market_cap_basic = df.market_cap_basic * 1E-6
df.premarket_volume = df.premarket_volume * 1E-6
df.float_shares_outstanding = df.float_shares_outstanding * 1E-6
df = df.rename(columns={
    "volume": "Volume (M)", 
    "market_cap_basic": "Market Cap (M)",
    "premarket_change": "Premarket %",
    "premarket_change_abs": "Premarket $",
    "premarket_volume": "Premarket Volume (M)",
    "float_shares_outstanding": "Float (M)"
    })
print(n_rows)

display(df)

In [ ]:
import datetime as dt
from download import *
from visualize import *

yf_df = download_from_yf("EBS", "1m", dt.date(2024,8,28), dt.date(2024,8,30))


In [ ]:
# pd.set_option('display.max_rows', 2500)
display(yf_df)

In [ ]:
import datetime as dt
from download import *
from visualize import *

polygon_df = download_from_polygon("EBS", "4h", dt.date(2024,8,28), dt.date(2024,8,30))
display(polygon_df)

In [ ]:
pd.set_option('display.max_rows', 3000)
merged_df = yf_df.merge(polygon_df, how='left')
display(merged_df)
pd.set_option('display.max_rows', 10)

In [ ]:
market_open = pd.to_datetime('09:30:00').time()
market_close = pd.to_datetime('16:00:00').time()

pre_post_data = polygon_df.loc[(polygon_df['Datetime'].dt.time <= market_open) | (polygon_df['Datetime'].dt.time >= market_close)]

# TODO: Lookup not by exact value, but by the closest timestamp in yf_df after the query timestamp. 
for index, row in pre_post_data.iterrows():
    yf_row = yf_df[yf_df['Datetime'] == row.Datetime]
    assert len(yf_row) <= 1, "Multiple of the same timestamps found."
    yf_df.loc[yf_row.index, 'Volume'] = row.Volume
    

# pre_post_data["closest_timestamp"] = pd.merge_asof(pre_post_data, yf_df, on='Datetime', direction='forward')['Datetime']

In [ ]:
import pandas_market_calendars as mcal
import pandas as pd
import datetime as dt

market_schedule = mcal.get_calendar('NYSE').schedule(start_date='2024-01-01', end_date='2024-12-31')
regular_business_days = market_schedule.loc[market_schedule['market_close'].dt.time >= pd.to_datetime('20:00:00').time()].index.to_list()

regular_business_days = [i.to_pydatetime().date() for i in regular_business_days]
print(regular_business_days)
print((dt.datetime.now().date() + dt.timedelta(days=1)) in regular_business_days)

In [ ]:
import yfinance as yf
import datetime as dt
import requests
import pandas as pd

interval = 'Daily' # '5min'
# url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval={interval}&outputsize=full&apikey=demo"
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=demo"
r = requests.get(url)
data = r.json()

# data = yf.download('IBM', interval='1d', start=dt.datetime(1914,1,1), end=dt.datetime.today(), prepost=True, progress=False)
# display(data)

df = pd.DataFrame(columns=["Datetime", 'Open', 'High', 'Low', 'Close', 'Volume'])
for timestamp, ohlcv in data[f"Time Series ({interval})"].items():
    df = pd.concat([pd.DataFrame([[dt.datetime.fromisoformat(timestamp),
                     ohlcv['1. open'],
                     ohlcv['2. high'],
                     ohlcv['3. low'],
                     ohlcv['4. close'],
                     ohlcv['5. volume']]], columns=df.columns), df], ignore_index=True)
display(df)

In [ ]:
from download import * 

df = download_and_save("EBS", "1m", save_to_pickle=False)

In [ ]:
pd.set_option('display.max_rows', 2500)
display(df)
pd.set_option('display.max_rows', 100)

In [ ]:
from bisect import bisect_left
import datetime as dt
from download import * 
for query_date in [dt.date(2024, 1, 1) + dt.timedelta(days=i) for i in range(365)]:
    result = most_recent_business_day(query_date)
    print(query_date, result)

In [ ]:
'''
References: 
https://www.projectpro.io/recipes/upload-files-to-google-drive-using-python
https://stackoverflow.com/questions/75454425/access-blocked-project-has-not-completed-the-google-verification-process
https://pythonhosted.org/PyDrive/oauth.html
'''
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

file = 'C:\\Users\\georg\\GitHub\\money-tree\\test_run.html'

gauth = GoogleAuth(settings_file='C:\\Users\\georg\\GitHub\\money-tree\\settings.yaml', http_timeout=30)
drive = GoogleDrive(gauth)
upload_file_list = [file] 
for upload_file in upload_file_list: 
    gfile = drive.CreateFile({'parents': [{'id': '1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q'}]}) # Read file and set it as the content of this instance. 
    gfile.SetContentFile(upload_file) 
    gfile.Upload() # Upload the file.

# list file
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1N_7YUWKB2ezW_d_fse719Xo_toFuqx0q')}).GetList() 
for file in file_list: 
    print('title: %s, id: %s' % (file['title'], file['id']))

# # download file
# for i, file in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
#     print('Downloading {} file from GDrive ({}/{})'.format(file['title'], i, len(file_list))) 
#     file.GetContentFile(file['title'])

In [ ]:
import sys
print(sys.platform)
import platform
print(platform.system())


In [ ]:
from download import * 
import os
test_pickle_path = pickle_filepath("NVDA", "1d")
print(test_pickle_path)
print(os.path.exists(test_pickle_path))


In [ ]:
import yfinance as yf
import datetime as dt
from download import * 

# data = download_from_polygon("NVDA", "1h", start=dt.date(2024, 9, 10), end = dt.date(2024, 9, 10))
# display(data)

data = download_from_yf("USDJPY=X", "1h", start=dt.date(2024, 9, 10), end = dt.date(2024, 9, 10))
display(data)


In [ ]:
from download import * 
data = download_and_save("NVDA", "1h")
display(data)

In [ ]:
import pickle
import os
import pandas as pd

filepath = "G:\\My Drive\\data\\QH_1h.pkl"
print(os.path.exists(filepath))
with open(filepath, 'rb') as f:
    data = pickle.load(f)
pd.set_option('display.max_rows', 2500)
display(data)
pd.set_option('display.max_rows', 100)



In [ ]:
from download import *

is_strictly_continuous(data['Datetime'], '4h')

In [2]:
from download import * 
(download_from_yf("CHSN", "1m", dt.date(2024,10,8), dt.date(2024,10,8)))


1 Failed download:
['CHSN']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1m 2024-10-08 -> 2024-10-09)')


$CHSN: possibly delisted; No price data found  (1m 2024-10-08 -> 2024-10-09)


,Datetime,Open,High,Low,Close,Volume


In [1]:
from download import * 
import datetime as dt
data = download_and_save("NVDA", "1h", dt.date(2024,10,17), dt.date(2024,10,17), save_to_pickle=False)
display(data)


Business days loaded!
Downloading:	NVDA	1h	2024-10-17	2024-10-17


,Datetime,Open,High,Low,Close,Volume
0,2024-10-17 04:00:00,138.430000,139.400000,137.300000,139.120000,1448593
1,2024-10-17 05:00:00,139.130000,139.320000,138.830000,138.850000,679409
2,2024-10-17 06:00:00,138.900000,139.330000,138.600000,139.300000,1512980
3,2024-10-17 07:00:00,139.300000,140.300000,139.200000,139.859000,2358342
4,2024-10-17 08:00:00,139.840000,140.230000,135.410000,139.770000,5982131
5,2024-10-17 09:00:00,139.790000,140.300000,132.785000,139.330000,72170977
6,2024-10-17 09:30:00,139.350006,140.889999,137.375000,139.600601,121786439
7,2024-10-17 10:30:00,139.604996,139.740005,138.389999,139.649994,40698586
8,2024-10-17 11:30:00,139.649994,140.251999,139.220001,139.460098,32335890
9,2024-10-17 12:30:00,139.464996,139.679993,138.893402,139.539902,21696482


In [6]:
empty_df = download_and_save("BBLG", interval="1m", start=dt.date.today(), end=dt.date.today())
display(empty_df)
data = data.merge(empty_df, how='outer')
display(data)

Downloading:	BBLG	1m	2024-10-18	2024-10-18


,Volume,Open,Close,High,Low,Datetime
0,0,1.91,1.91,1.91,1.91,2024-10-18 04:20:00
0,640,1.94,1.94,1.94,1.93,2024-10-18 05:00:00
1,1000,1.95,1.95,1.95,1.95,2024-10-18 05:20:00
2,100,1.94,1.94,1.94,1.94,2024-10-18 05:21:00
3,1000,1.94,1.95,1.95,1.94,2024-10-18 05:22:00
...,...,...,...,...,...,...
515,500,2.09,2.09,2.09,2.09,2024-10-18 19:40:00
516,4890,2.13,2.16,2.16,2.13,2024-10-18 19:44:00
517,400,2.16,2.16,2.16,2.16,2024-10-18 19:49:00
506,0,2.10,2.10,2.10,2.10,2024-10-18 19:51:33


,Datetime,Open,High,Low,Close,Volume
0,2024-10-17 04:00:00,138.43,139.40,137.30,139.120,1448593
1,2024-10-17 05:00:00,139.13,139.32,138.83,138.850,679409
2,2024-10-17 06:00:00,138.90,139.33,138.60,139.300,1512980
3,2024-10-17 07:00:00,139.30,140.30,139.20,139.859,2358342
4,2024-10-17 08:00:00,139.84,140.23,135.41,139.770,5982131
...,...,...,...,...,...,...
581,2024-10-18 19:40:00,2.09,2.09,2.09,2.090,500
582,2024-10-18 19:44:00,2.13,2.16,2.13,2.160,4890
583,2024-10-18 19:49:00,2.16,2.16,2.16,2.160,400
584,2024-10-18 19:51:33,2.10,2.10,2.10,2.100,0


In [ ]:
# one time fix data
from download import *
# affected_tickers = ['SOBR', 'HEPS', 'DRUG', 'JDZG', 'PEGY', 'AKAN', 'TNON', 'TVGN', 'SLRX', 'MGRX', "PRME", "GFI", "RTO"]
# for ticker in affected_tickers:
#     os.remove(pickle_filepath(ticker, "1h"))

# # add it back
# for ticker in affected_tickers:
#     download_and_save(ticker, "1h", dt.date(2024,10,18), dt.date(2024,10,18), save_to_pickle=True)

# affected_tickers = ["QQQ", "SPY", "VOO", "NVDA", "MSFT"]
# for ticker in affected_tickers:
#     os.remove(pickle_filepath(ticker, "1h"))

# # add it back
# for ticker in affected_tickers:
#     download_and_save(ticker, "1h", save_to_pickle=True)

In [ ]:
from download import *
test_ticker = "BBLG"
test_interval = "5m"
df = download_and_save(test_ticker, interval=test_interval, start=dt.date(2024,10,13), end=dt.date(2024,10,14), save_to_pickle=False)
pd.set_option('display.max_rows', 2500)
display(df)
pd.set_option('display.max_rows', 100)
is_df_continuous, missing_times = is_strictly_continuous(df['Datetime'], test_interval)
print(is_df_continuous, missing_times)
is_df_continuous, missing_times = is_loosely_continuous(df['Datetime'], test_interval)
print(is_df_continuous, missing_times)

In [ ]:
from download import *
big_names = {"QQQ", "SPY", "VOO", "NVDA", "MSFT"}
affected_tickers = set()
folder = os.path.join(BASE_DIR, "data")
for f in os.listdir(folder):
    if "_1h.pkl" in f:
        affected_tickers.add(f.removesuffix("_1h.pkl"))

affected_tickers.difference_update(big_names)
repair_1h_info = dict()
to_delete = []
for ticker in affected_tickers:
    df = get_downloaded_data_or_none(ticker, "1h")
    if df is None or dt.time(hour=10, minute=0) in df['Datetime'].dt.time.unique(): 
        to_delete.append(ticker)
    else:
        repair_1h_info[ticker] = df['Datetime'].dt.date.unique()
print(repair_1h_info.keys())

# print(to_delete)
# for ticker in to_delete:
#     os.remove(pickle_filepath(ticker, "1h"))


In [ ]:
# for ticker, dates in repair_1h_info.items():
#     os.remove(pickle_filepath(ticker, "1h"))
#     for date in dates:
#         download_and_save(ticker, "1h", date, date)

In [43]:
from download import *
# print(dt.date.today()- dt.timedelta(days=60))
# data = yf.download("NVDA", interval="5m", start=dt.date(2024, 8, 23), end=dt.date.today(), prepost=True)
# print(sorted(list(set(data.index.date))))

print(dt.date.today()- dt.timedelta(days=730))
interval = "1h"
start = dt.date(2024, 10,17)
end = dt.date(2024, 10,18)
interval_abbrev_dict = {"m": "minute", "h": "hour", "d": "day"}
interval_arg = interval[:-1] + "/" + interval_abbrev_dict[interval[-1]]
polygon_url = f"https://api.polygon.io/v2/aggs/ticker/NVDA/range/{interval_arg}/{start.isoformat()}/{end.isoformat()}?adjusted=true&sort=asc&limit=50000&apiKey={os.environ['POLYGON_API_KEY']}"
r = requests.get(polygon_url)
print(r.json())
print(dt.datetime.utcfromtimestamp(1666598400).strftime('%Y-%m-%d %H:%M:%S'))

2022-10-22
{'ticker': 'NVDA', 'queryCount': 1916, 'resultsCount': 32, 'adjusted': True, 'results': [{'v': 1448593.0, 'vw': 139.0433, 'o': 138.2, 'c': 139.18, 'h': 139.4, 'l': 137.05, 't': 1729152000000, 'n': 18142}, {'v': 679409, 'vw': 138.9565, 'o': 139.18, 'c': 138.9, 'h': 139.32, 'l': 138.6, 't': 1729155600000, 'n': 7880}, {'v': 1512980.0, 'vw': 139.6743, 'o': 138.9, 'c': 140.08, 'h': 140.3, 'l': 138.9, 't': 1729159200000, 'n': 17847}, {'v': 2358342.0, 'vw': 139.8543, 'o': 140.07, 'c': 139.95, 'h': 140.09, 'l': 139.56, 't': 1729162800000, 'n': 20823}, {'v': 5982131.0, 'vw': 139.5221, 'o': 139.94, 'c': 139.94, 'h': 140.065, 'l': 135.41, 't': 1729166400000, 'n': 45219}, {'v': 72170977.0, 'vw': 139.4627, 'o': 139.94, 'c': 138.865, 'h': 140.89, 'l': 137.65, 't': 1729170000000, 'n': 470845}, {'v': 62630410.0, 'vw': 138.8939, 'o': 138.88, 'c': 139.12, 'h': 139.75, 'l': 137.3648, 't': 1729173600000, 'n': 423853}, {'v': 32071604.0, 'vw': 139.54, 'o': 139.13, 'c': 140.104, 'h': 140.13, 'l': 

C:\Users\georg\AppData\Local\Temp\ipykernel_32148\2650934983.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print(dt.datetime.utcfromtimestamp(1666598400).strftime('%Y-%m-%d %H:%M:%S'))


In [44]:
from download import *
total_df = pd.DataFrame(columns=["Datetime", 'Open', 'High', 'Low', 'Close', 'Volume'])  
total_df = total_df.drop_duplicates("Datetime").sort_values(by="Datetime")